# Relocations and Trajectories

## Setup

### Ecoscope

In [ ]:
ECOSCOPE_RAW = "https://raw.githubusercontent.com/wildlife-dynamics/ecoscope/master"

In [ ]:
%pip install \
    'ecoscope[analysis,mapping,plotting] @ git+https://github.com/wildlife-dynamics/ecoscope@v2.2.1' &> /dev/null

In [ ]:
import os
import sys

import geopandas as gpd
import pandas as pd
import shapely

import ecoscope
from ecoscope.analysis.astronomy import is_night

ecoscope.init()

### Google Drive Setup

In [ ]:
output_dir = "Ecoscope-Outputs"

if "google.colab" in sys.modules:
    from google.colab import drive

    drive.mount("/content/drive/", force_remount=True)
    output_dir = os.path.join("/content/drive/MyDrive/", output_dir)

os.makedirs(output_dir, exist_ok=True)

## `Relocations`

### Read in sample data in MoveBank format

In [ ]:
ecoscope.io.download_file(
    f"{ECOSCOPE_RAW}/tests/sample_data/vector/movebank_data.csv",
    os.path.join(output_dir, "movebank_data.csv"),
)

data = pd.read_csv(os.path.join(output_dir, "movebank_data.csv"))

### Make Data Timezone Aware and Sort by Time

In [ ]:
data["timestamp"] = pd.to_datetime(data["timestamp"], utc=True)
data.sort_values(["timestamp"], inplace=True)

### Create GeoDataFrame Using Lat/Lon Point Coordinates

In [ ]:
gdf = gpd.GeoDataFrame(
    data,
    geometry=gpd.points_from_xy(x=data["location-long"], y=data["location-lat"]),
    crs=4326,
)

Display columns:

In [ ]:
gdf.columns

### Create `Relocations` from Specified Time, Subject Id, and Observation Id

In [ ]:
relocs = ecoscope.Relocations.from_gdf(
    gdf,
    groupby_col="individual-local-identifier",
    time_col="timestamp",
    uuid_col="event-id",
)

When the dataframe becomes a `Relocations` it will gain `fixtime`, `groupby_col`, `junk_status` columns that are expected by Ecoscope analyses

In [ ]:
relocs.gdf.columns

### Optionally `drop` Non-Essential Columns

In [ ]:
relocs.gdf = relocs.gdf[["groupby_col", "fixtime", "junk_status", "geometry"]]

In [ ]:
relocs.gdf

Which unique subjects are contained in the `groupby_col` column?

In [ ]:
relocs.gdf["groupby_col"].unique()

### Visualize Data

In [ ]:
relocs.gdf[["groupby_col", "geometry"]].explore()

### Mark Points as Junk Based on Coordinates

In [ ]:
coord_filter = ecoscope.base.RelocsCoordinateFilter(
    min_x=-5,
    max_x=1,
    min_y=12,
    max_y=18,
    filter_point_coords=[[180, 90], [0, 0]],
)
relocs.apply_reloc_filter(coord_filter, inplace=True)

Count values marked as junk:

In [ ]:
relocs.gdf["junk_status"].value_counts()

### Mark Points as Junk Based on Speed

In [ ]:
speed_filter = ecoscope.base.RelocsSpeedFilter(max_speed_kmhr=4.0)
relocs.apply_reloc_filter(speed_filter, inplace=True)

In [ ]:
relocs.gdf["junk_status"].value_counts()

### Remove Flagged Fixes

In [ ]:
relocs.remove_filtered(inplace=True)

### Remove Duplicate Fixes

In [ ]:
print(relocs.gdf.duplicated(subset=["fixtime", "geometry"]).any())

relocs.gdf = relocs.gdf.drop_duplicates(subset=["fixtime", "geometry"])

Visualize:

In [ ]:
relocs.gdf["geometry"].explore()

### Determine whether each relocation is a day or night

In [ ]:
relocs.gdf["is_night"] = is_night(relocs.gdf.geometry, relocs.gdf.fixtime)

### Break Relocations Geometry Into X and Y

In [ ]:
relocs.gdf["lat"] = relocs.gdf["geometry"].y
relocs.gdf["lon"] = relocs.gdf["geometry"].x

### Convert to UTM and Back to WGS 84

In [ ]:
relocs.gdf.to_crs(relocs.gdf.estimate_utm_crs(), inplace=True)
relocs.gdf["extra__northing"] = relocs.gdf["geometry"].y
relocs.gdf["extra__easting"] = relocs.gdf["geometry"].x
relocs.gdf.to_crs(4326, inplace=True)

In [ ]:
relocs.gdf.columns

### Export to Compressed CSV

In [ ]:
relocs.gdf.to_csv(
    os.path.join(output_dir, "relocs_clean.csv.zip"),
    header=True,
    index=True,
    compression="zip",
)

## `Trajectory`



### Read in `Relocations`

`relocs` from the previous steps can be used instead of redefining it here.

In [ ]:
ecoscope.io.download_file(
    f"{ECOSCOPE_RAW}/tests/sample_data/vector/er_relocs.csv.zip",
    os.path.join(output_dir, "er_relocs.csv.zip"),
)

data = pd.read_csv(os.path.join(output_dir, "er_relocs.csv.zip"), header=0, index_col=0)

#### Parse Geometry from WKT

In [ ]:
gdf = gpd.GeoDataFrame(data, geometry=data["geometry"].map(lambda x: shapely.wkt.loads(x)), crs=4326)

In [ ]:
relocs = ecoscope.Relocations.from_gdf(gdf)

### Create `Trajectory` from `Relocations`

In [ ]:
traj = ecoscope.Trajectory.from_relocations(relocs)

In [ ]:
traj.gdf

In [ ]:
traj.gdf.columns

Visualize:

In [ ]:
traj.gdf["geometry"].explore()

### Filter Segments

In [ ]:
traj_seg_filter = ecoscope.base.TrajSegFilter(
    min_length_meters=0.0,
    max_length_meters=float("inf"),
    min_time_secs=0.0,
    max_time_secs=4 * 60 * 60,
    min_speed_kmhr=0.0,
    max_speed_kmhr=8.0,
)
traj.apply_traj_filter(traj_seg_filter, inplace=True)

In [ ]:
traj.gdf["junk_status"].value_counts()

In [ ]:
traj.remove_filtered(inplace=True)

Visualize:

In [ ]:
traj.gdf["geometry"].explore()

### Optionally Drop Non-Essential Columns

In [ ]:
traj.gdf = traj.gdf[
    [
        "groupby_col",
        "segment_start",
        "segment_end",
        "timespan_seconds",
        "dist_meters",
        "speed_kmhr",
        "heading",
        "geometry",
    ]
]

### Calculate Summary Statistics

In [ ]:
traj.gdf.groupby(by=["groupby_col"]).apply(pd.DataFrame.describe)

### Export to GeoPackage for ArcGIS or QGIS

In [ ]:
traj.gdf.to_file(os.path.join(output_dir, "traj.gpkg"))

### Convert a Trajectory object to a Relocation object

In [ ]:
new_relocs = traj.to_relocations()

### Upsampling

In [ ]:
upsampled_relocs = traj.upsample("180S")

### Downsampling, with interpolation

In [ ]:
downsampled_relocs_int = traj.downsample("10800S", interpolation=True)

### Downsampling, without interpolation

In [ ]:
downsampled_relocs_noint = traj.downsample("10800S", tolerance="900S")